<a href="https://colab.research.google.com/github/ArturHugo/PLN-2022-1/blob/main/Atividade03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atividade 03 - Recuperação de textos
Grupo:

- Artur Hugo cunha Pereira, 180030400
- Gabriel da Silva Corvino Nogueira, 180113330 


In [91]:
import re
import pandas as pd
import nltk
import numpy as np

In [92]:
# NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Corpus Reuters

In [93]:
nltk.download('reuters')
from nltk.corpus import reuters

cats = reuters.categories()
print("Reuters has %d categories:\n%s" % (len(cats), cats))

fileids = reuters.fileids()

categories = []
text = []

for file in fileids:
    categories.append(reuters.categories(file))
    text.append(reuters.raw(file))

df = pd.DataFrame({'ids':fileids, 'categories':categories, 'text':text})
df

[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


Reuters has 90 categories:
['acq', 'alum', 'barley', 'bop', 'carcass', 'castor-oil', 'cocoa', 'coconut', 'coconut-oil', 'coffee', 'copper', 'copra-cake', 'corn', 'cotton', 'cotton-oil', 'cpi', 'cpu', 'crude', 'dfl', 'dlr', 'dmk', 'earn', 'fuel', 'gas', 'gnp', 'gold', 'grain', 'groundnut', 'groundnut-oil', 'heat', 'hog', 'housing', 'income', 'instal-debt', 'interest', 'ipi', 'iron-steel', 'jet', 'jobs', 'l-cattle', 'lead', 'lei', 'lin-oil', 'livestock', 'lumber', 'meal-feed', 'money-fx', 'money-supply', 'naphtha', 'nat-gas', 'nickel', 'nkr', 'nzdlr', 'oat', 'oilseed', 'orange', 'palladium', 'palm-oil', 'palmkernel', 'pet-chem', 'platinum', 'potato', 'propane', 'rand', 'rape-oil', 'rapeseed', 'reserves', 'retail', 'rice', 'rubber', 'rye', 'ship', 'silver', 'sorghum', 'soy-meal', 'soy-oil', 'soybean', 'strategic-metal', 'sugar', 'sun-meal', 'sun-oil', 'sunseed', 'tea', 'tin', 'trade', 'veg-oil', 'wheat', 'wpi', 'yen', 'zinc']


,ids,categories,text
0,test/14826,[trade],ASIAN EXPORTERS FEAR DAMAGE FROM U.S.-JAPAN RI...
1,test/14828,[grain],CHINA DAILY SAYS VERMIN EAT 7-12 PCT GRAIN STO...
2,test/14829,"[crude, nat-gas]",JAPAN TO REVISE LONG-TERM ENERGY DEMAND DOWNWA...
3,test/14832,"[corn, grain, rice, rubber, sugar, tin, trade]",THAI TRADE DEFICIT WIDENS IN FIRST QUARTER\n ...
4,test/14833,"[palm-oil, veg-oil]",INDONESIA SEES CPO PRICE RISING SHARPLY\n Ind...
...,...,...,...
10783,training/999,"[interest, money-fx]",U.K. MONEY MARKET SHORTAGE FORECAST REVISED DO...
10784,training/9992,[earn],KNIGHT-RIDDER INC &lt;KRN> SETS QUARTERLY\n Q...
10785,training/9993,[earn],TECHNITROL INC &lt;TNL> SETS QUARTERLY\n Qtly...
10786,training/9994,[earn],NATIONWIDE CELLULAR SERVICE INC &lt;NCEL> 4TH ...


# Preprocessamento dos dados

In [94]:
# Helper functions
def tokenize(string):
  # Remove special characters
  res = re.sub(r'&lt;|\\n|>', ' ', string.lower())
  # Runs through nltk tokenizer to finish tokenization
  res = nltk.tokenize.word_tokenize(res)
  return res

def filter_stopwords(word_array, stop_list):
  # If word is in stop_list it is filtered out
  res = [word for word in word_array if word not in stop_list]
  return res

In [95]:
texts_df = df['text']
texts_df = texts_df.sample(frac=0.1)
texts_df

9154     NAPCO INTERNATIONAL &lt;NPCO> 4TH QTR OPER NET...
8982     AGENCY RENT-A-CAR &lt;AGNC> SETS STOCK DIVIDEN...
9788     MANHATTAN NATIONAL CORP &lt;MLC> 4TH QTR LOSS\...
7503     K-TRON INTERNATIONAL INC &lt;KTII> 4TH QTR NET...
825      GIANT BAY &lt;GBYLF> IN IDAHO GOLD VENTURE\n  ...
                               ...                        
10133    ALEXANDER'S &lt;ALX> 2ND QTR ENDS FEB 27 NET\n...
6171     LANDMARK SAVINGS &lt;LSA> COMPLETES OFFICE SAL...
2412     TRANSAMERICA &lt;TA> TO BUY SEDGWICK &lt;SDWK....
10090    FED'S HELLER URGES JAPANESE TO BUY U.S. GOODS\...
1987     U.S. BUSINESS LOANS FALL 896 MLN DLRS IN JUNE ...
Name: text, Length: 1079, dtype: object

In [96]:
texts_df = texts_df.apply(tokenize)
texts_df

9154     [napco, international, npco, 4th, qtr, oper, n...
8982     [agency, rent-a-car, agnc, sets, stock, divide...
9788     [manhattan, national, corp, mlc, 4th, qtr, los...
7503     [k-tron, international, inc, ktii, 4th, qtr, n...
825      [giant, bay, gbylf, in, idaho, gold, venture, ...
                               ...                        
10133    [alexander, 's, alx, 2nd, qtr, ends, feb, 27, ...
6171     [landmark, savings, lsa, completes, office, sa...
2412     [transamerica, ta, to, buy, sedgwick, sdwk.l, ...
10090    [fed, 's, heller, urges, japanese, to, buy, u....
1987     [u.s., business, loans, fall, 896, mln, dlrs, ...
Name: text, Length: 1079, dtype: object

In [97]:
# Remove stop words
stop_list = nltk.corpus.stopwords.words('english')
stop_list.extend(['.', '\'\'', '``', '(', ')', ',', '\'s']) # Additional stopwords
texts_df = texts_df.apply(lambda text: filter_stopwords(text, stop_list))
texts_df

9154     [napco, international, npco, 4th, qtr, oper, n...
8982     [agency, rent-a-car, agnc, sets, stock, divide...
9788     [manhattan, national, corp, mlc, 4th, qtr, los...
7503     [k-tron, international, inc, ktii, 4th, qtr, n...
825      [giant, bay, gbylf, idaho, gold, venture, gian...
                               ...                        
10133    [alexander, alx, 2nd, qtr, ends, feb, 27, net,...
6171     [landmark, savings, lsa, completes, office, sa...
2412     [transamerica, ta, buy, sedgwick, sdwk.l, unit...
10090    [fed, heller, urges, japanese, buy, u.s., good...
1987     [u.s., business, loans, fall, 896, mln, dlrs, ...
Name: text, Length: 1079, dtype: object

In [98]:
# Stemming
stemmer = nltk.stem.porter.PorterStemmer()
texts_df = texts_df.apply(lambda text: [stemmer.stem(word) for word in text])
texts_df

9154     [napco, intern, npco, 4th, qtr, oper, net, ope...
8982     [agenc, rent-a-car, agnc, set, stock, dividend...
9788     [manhattan, nation, corp, mlc, 4th, qtr, loss,...
7503     [k-tron, intern, inc, ktii, 4th, qtr, net, ope...
825      [giant, bay, gbylf, idaho, gold, ventur, giant...
                               ...                        
10133    [alexand, alx, 2nd, qtr, end, feb, 27, net, sh...
6171     [landmark, save, lsa, complet, offic, sale, la...
2412     [transamerica, ta, buy, sedgwick, sdwk.l, unit...
10090    [fed, heller, urg, japanes, buy, u.s., good, m...
1987     [u.s., busi, loan, fall, 896, mln, dlr, june, ...
Name: text, Length: 1079, dtype: object

# Representações: BoW e TF-IDF

In [99]:
# Corpus and vocabulary

corpus = []
for text in texts_df:
  for word in text:
    corpus.append(word)

vocab = set(sorted(corpus))

In [100]:
# Counting Bag of Words

bow_count = {}
for i, text in enumerate(texts_df.values):
  bow_count[df['ids'][i]] = dict()
  for word in text:
    if word in bow_count[df['ids'][i]]:
      bow_count[df['ids'][i]][word] += 1
    else:
      bow_count[df['ids'][i]][word] = 1

bow_count_df = pd.DataFrame().from_records(bow_count).fillna(0).T.astype(int)
bow_count_df

,napco,intern,npco,4th,qtr,oper,net,shr,38,ct,...,"1,488,000",lsa,whitehal,parkval,sedgwick,sdwk.l,thame,reinsur,'d,896
test/14826,2,2,1,1,1,9,5,2,1,2,...,0,0,0,0,0,0,0,0,0,0
test/14828,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
test/14829,0,0,0,1,2,4,3,2,0,4,...,0,0,0,0,0,0,0,0,0,0
test/14832,0,1,0,1,1,4,5,2,0,2,...,0,0,0,0,0,0,0,0,0,0
test/14833,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
test/16717,0,0,0,0,2,0,3,4,0,9,...,1,0,0,0,0,0,0,0,0,0
test/16718,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,0,0,0,0,0,0
test/16719,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,3,1,4,1,0,0
test/16723,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [101]:
# TF-IDF

document_freq = {}
for word in vocab:
  document_freq[word] = 0
  for text, word_counts in bow_count.items():
    if word in word_counts:
      document_freq[word] += 1

tf_idf  = {}
n_texts = len(texts_df.values)
for text_id, word_counts in bow_count.items():
  n_words = sum(word_counts.values())
  tf_idf[text_id] = {}
  for word, count in word_counts.items():
    tf  = count/n_words
    idf = np.log(n_texts/document_freq[word])
    tf_idf[text_id][word] = tf*idf

tf_idf_df = pd.DataFrame().from_records(tf_idf).fillna(0).T
tf_idf_df

,napco,intern,npco,4th,qtr,oper,net,shr,38,ct,...,"1,488,000",lsa,whitehal,parkval,sedgwick,sdwk.l,thame,reinsur,'d,896
test/14826,0.147027,0.046238,0.073514,0.030772,0.017901,0.190142,0.069700,0.032911,0.047357,0.027954,...,0.000000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
test/14828,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
test/14829,0.000000,0.000000,0.000000,0.036542,0.042515,0.100353,0.049662,0.039081,0.000000,0.066390,...,0.000000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
test/14832,0.000000,0.022878,0.000000,0.030452,0.017714,0.083627,0.068974,0.032568,0.000000,0.027662,...,0.000000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
test/14833,0.000000,0.000000,0.000000,0.000000,0.000000,0.028672,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
test/16717,0.000000,0.000000,0.000000,0.000000,0.040014,0.000000,0.046740,0.073565,0.000000,0.140590,...,0.082162,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
test/16718,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.21163,0.21163,0.21163,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
test/16719,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018652,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00000,0.00000,0.29509,0.098363,0.393453,0.098363,0.000000,0.000000
test/16723,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.088402,0.000000


# Recuperação de informação

In [102]:
# Cosine similarity
def cosine_similarity(v1, v2):
  return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [103]:
# Calcula similaride de uma notícia com todas as demais notícias
def similarity_ranking(text_id, texts_df, sim_function):
  '''
    text_id:      row's id of the text representation in the dataframe.
    texts_df:     dataframe with the texts to rank against.
    sim_function: similarity function to use in the comparisons.

    Calculate the similarity between the text in row with name text_id and 
    the remaining texts in the dataframe texts_df using sim_function.

    Returns a list o text_ids in order of most to least similar.
  '''

  # Calculate similarities
  query_text = texts_df.loc[text_id].values
  similarities_df = texts_df.apply(
      lambda text: sim_function(text.values, query_text),
      axis=1)
  
  # Remove the query text's similarity with itself
  similarities_df = similarities_df.drop(text_id)

  # Sort from most to least similar
  similarities_df = similarities_df.sort_values(ascending=False)

  # Return the similarity ranking with the id of the texts
  return similarities_df.index.to_list()

In [104]:
# Ranking de similaridade usando BoW
bow_count_ranking = bow_count_df.apply(
    lambda text: similarity_ranking(text.name, bow_count_df, cosine_similarity),
    axis=1).to_dict()

# Ranking de similaridades usando TF-IDF
tf_idf_ranking = tf_idf_df.apply(
    lambda text: similarity_ranking(text.name, tf_idf_df, cosine_similarity),
    axis=1).to_dict()

In [105]:
# Limitando o rankings ao top 10
bow_count_top = bow_count_ranking
for text_id in bow_count_top:
  bow_count_top[text_id] = bow_count_top[text_id][0:10]

tf_idf_top = tf_idf_ranking
for text_id in tf_idf_top:
  tf_idf_top[text_id] = tf_idf_top[text_id][0:10]

In [106]:
# Confere se duas listas tem elementos em comum
def common(list_a, list_b):
  set_a = set(list_a)
  set_b = set(list_b)
  if set_a & set_b:
    return True
  else:
    return False

In [107]:
# Verifica a acurácia do ranking de um dicionário de similaridade
def top_accuracies(ranking_dict, original_df):
  '''
    ranking_dict: dictionary of similarity rankings for each text in dataframe.
    original_df:  original dataframe to check categories of the rankings.

    Calculate the how many elements of the top ranking had a category in common
    with their respective query text.

    Returns a dictionary with text ids for keys and their respective accuracy
    for values.
  '''

  accuracies_dict = {}
  for text_id in bow_count_top:
    accuracies_dict[text_id] = 0
    text_categories = original_df[original_df['ids'] == text_id]['categories'].values[0]
    top_ids = bow_count_top[text_id]
    for top_id in top_ids:
      top_categories = original_df[original_df['ids'] == top_id]['categories'].values[0]
      if common(text_categories, top_categories):
        accuracies_dict[text_id] += 1
    accuracies_dict[text_id] /= len(bow_count_top[text_id])

  return accuracies_dict

In [108]:
# Verificar acurácia dos top 10
bow_count_top_accuracies = top_accuracies(bow_count_top, df)

tf_idf_top_accuracies = top_accuracies(tf_idf_top, df)

In [109]:
# Acurácias médias para comparar
bow_count_avg_accuracy = np.average(list(bow_count_top_accuracies.values()))
print(f'Average accuracy for BoW:    {bow_count_avg_accuracy}')
tf_idf_avg_accuracy = np.average(list(tf_idf_top_accuracies.values()))
print(f'Average accuracy for TF-IDF: {tf_idf_avg_accuracy}')

Average accuracy for BoW:    0.22659870250231695
Average accuracy for TF-IDF: 0.22659870250231695
